# Final Report and Summary of Analysis - King County Housing Sale Prices

## CONTENTS

1. Purpose
2. Summary of Workflow 
3. Importing
4. Square Footage Data - Analysis
5. Porch Data - Analysis
6. Waterfront Data - Analysis
7. Nuisance Data - Analysis
8. Final Model Creation
9. Recommendations for Real World Application
10. Next Steps

## 1. Purpose
This project is aimed to help homeowners and potential homeowners in King County understand how specific features affect home sale prices in the area. The data we will use can be downloaded from here: 
https://info.kingcounty.gov/assessor/DataDownload/default.aspx.
Through our analysis our end goal is to build a statistical model that is representative of the real world data. 

In summary, the report that follows addresses following questions:

    1. Does higher square footage increase home sale price?
    2. Does having a porch increases home sale price?
    3. Does having a waterfront increase home sale prices?
    4. Does the presence of a nuisance (power lines, traffic noise, airport noise) lower home sale prices?
    5. Using this data, can a statistical model be built that accurately represents housing prices in King County?

## 2. Summary of Workflow

## 3. Importing
* Libraries
* Functions
* Data

## 4. Square Footage Data - Analysis

## 5. Porch Data - Analysis

## 6. Waterfront Data - Analysis

## 7. Nuisance Data - Analysis

## 8. Final Model Creation

## 9. Recommendations for Real World Application

## 10. Next Steps